In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as tud
from collections import Counter, defaultdict
import operator
import os, math
import numpy as np
import random
import copy
import pandas as pd

In [45]:
import pandas as pd

In [46]:
# set the random seeds so the experiments can be replicated exactly
seed = 30255
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [47]:
#data = pd.read_csv('Data/data_prepr.csv')
data = pd.read_csv('Data/data_prepr_final.csv')

data = data[['tagged_rel', 'tagged_str']]

data.tagged_str = data.tagged_str.str.replace('AGGRESOR', ' AGGRESOR ')
data.tagged_str = data.tagged_str.str.replace('VICTIM', ' VICTIM ')

In [48]:
data.shape

(34556, 2)

In [49]:
def split_data(data):
    # reset index
    data_idx = data.reset_index()
    
    # subsetting in .75 train and even random split the other
    dev_test_size = round(len(data_idx)*0.25)
    tr_df = data_idx[dev_test_size:]
    dev_test_df = data_idx[:dev_test_size]
    ran_idx = np.random.choice(dev_test_size, round(dev_test_size/2), replace=False)
    dev_df = dev_test_df.iloc[ran_idx]
    test_df = dev_test_df.iloc[~ran_idx]
    
    #shuffling training so it doesn't see similar cases one after the other 
    tr_df = tr_df.sample(frac=1).reset_index(drop=True)
        
    # make them lists
    train_l = tr_df.values.tolist()
    dev_l = dev_df.values.tolist()
    test_l = test_df.values.tolist()
    all_l = data_idx.values.tolist()

    return train_l, dev_l, test_l, all_l

In [50]:
train_list, dev_list, test_list, all_list = split_data(data)

In [52]:
### CODE FOR PROCESSING THE TEXT FOR INPUTTING INTO NEURAL NETWORK 
VOCAB_SIZE = 5000

def remove_stopwords(l):
    STOP  = ['ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once', 'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for', 'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is', 's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until', 'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were', 'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above', 'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before', 'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then', 'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under', 'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which', 'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against', 'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'] 
    l_clean = []
    for i in STOP: 
        if i in l:
            l_clean.append(i)
    return l_clean 

def word_tokenize(s):
    split_l = s.lower().replace('.', '').replace(',', '').replace(';', '').replace(':', '').replace('!', '').replace('?', '').replace('(', '').replace(')', '').split()
    #clean_l = remove_stopwords(split_l)
    #return clean_l
    
    return split_l

class textModel:
    def __init__(self, data):
        # Vocabulary is a set that stores every word seen in the 
        # training data
        self.vocab_counts = Counter([word for ix, label, content in data 
                              for word in word_tokenize(content)]
                            ).most_common(VOCAB_SIZE-1)
        # word to index mapping
        self.word_to_idx = {k[0]: v+1 for v, k in 
                            enumerate(self.vocab_counts)}
        # all the unknown words will be mapped to index 0
        self.word_to_idx["UNK"] = 0 
        self.idx_to_word = {v:k for k, v in self.word_to_idx.items()}

        self.vocab = set(self.word_to_idx.keys())
        self.vocab_size = len(self.vocab) 
        
        
    def train_model(self, data):
        '''
        Train the model with the provided training data
        '''
        raise NotImplementedError
        
    def classify(self, data):
        '''
        Classify the documents with the model
        '''
        raise 
        
class labels:
    def __init__(self, data):
        # Vocabulary is a set that stores every word seen in the 
        # training data
        self.label_counts = list(Counter([label for ix, label, content in data]).items())
        # word to index mapping
        self.label_to_idx = {k[0]: v+1 for v, k in 
                            enumerate(self.label_counts)}
        # all the unknown words will be mapped to index 0
        self.idx_to_label = {v:k for k, v in self.label_to_idx.items()}

        self.labels = set(self.label_to_idx.keys())
        self.labels_size = len(self.labels) + 1

        
class TextClassificationDataset(tud.Dataset):
    '''
    PyTorch provides a common dataset interface. 
    See https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    The dataset encodes documents into indices. 
    With the PyTorch dataloader, you can easily get batched data for 
    training and evaluation. 
    '''
    def __init__(self, data, labd):
        
        self.data = data
        self.vocab_size = VOCAB_SIZE
        self.tm = textModel(data)
        self.labs = labd

        
    def __len__(self):
        return len(self.data)
    
    def wordToTensor(self, word):
        tensor = torch.zeros(1, VOCAB_SIZE, dtype=torch.long)
        tensor[0][tm.word_to_idx[word]] = 1
        return tensor

    def lineToTensor(self, textstr):
        text = word_tokenize(textstr)
        to_ix = tm.word_to_idx
        tensor = autograd.Variable(torch.LongTensor([to_ix[w] if w in to_ix else to_ix['UNK']for w in text]))
        return tensor
    
    def targetToTensor(self, lab): 

        label_to_ix = self.labs.label_to_idx
        l = label_to_ix [lab]
        target_idx = torch.tensor([[l]])
        return target_idx
    
    
    def __getitem__(self, idx):        
        datum = self.data[idx]
        label = self.targetToTensor(datum[1])
        item = self.lineToTensor(datum[2])
        
        return item, label


In [94]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import torch.autograd as autograd

# https://github.com/claravania/lstm-pytorch/blob/master/model.py
torch.manual_seed(1)

class LSTMRNN(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(LSTMRNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,batch_first=True, bidirectional=False)
        self.hidden2cat = nn.Linear(hidden_dim, output_size)
        self.hidden = self.initHidden()
        self.optimizer = torch.optim.Adam(self.parameters())

        # From HW
        self.loss_fn = nn.NLLLoss(size_average=False)
        self.num_epochs = 8
        self.best_model = {}
        
    def initHidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)),
                    autograd.Variable(torch.zeros(1, 1, self.hidden_dim)))

    def forward(self, sentence):
        #print(batch.size(-1))
        embeds = self.word_embeddings(sentence)
        # embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        # ht is the last hidden state of the sequences
        out_space = self.hidden2cat(self.hidden[0][-1])
        #out_space = self.hidden2cat(lstm_out)
        output = F.log_softmax(out_space)
        
        return output
    

    def train_epoch(self, train_data, l):
        '''
        '''
        data = TextClassificationDataset(train_data, l)
        data = tud.DataLoader(data, batch_size = 1, shuffle = True)
        # Forward pass
            
        for sent, targets in data: 
            self.zero_grad() 
            self.hidden = self.initHidden()
            pred = self.forward(sent)
            
            # Calculate loss with prediction and target labels
            loss = self.loss_fn(pred, targets[0][0])

            # Backward pass
            loss.backward()
            
            # Apply optimizer step. 
            self.optimizer.step()
            
    def classify(self, val_data, l):
        '''
        This function classifies/predicts documents into their categories. 
        '''
        classif = []
        cuml_loss = 0
        
        data = TextClassificationDataset(val_data, l)
        data = tud.DataLoader(data, batch_size = 1, shuffle = True)
        with torch.no_grad():
            for sent, targets in data: 
                self.hidden = self.initHidden()
                pred = self.forward(sent)

                # Calculate loss with prediction and target labels
                loss = self.loss_fn(pred, targets[0][0])
                cuml_loss += loss

                # fingin the max label for accuracy
                pred_idx = ((torch.topk(pred, 1)[1])[0]).item()
                
                pred_rel = l.idx_to_label[pred_idx]

                # storing into output
                targ_rel = l.idx_to_label[(targets[0][0].item())]
                classif.append((targ_rel, sent, pred_rel))

        return classif, cuml_loss            
                
                
    def evaluate_classifier_accuracy(self, classif_dat):
        '''
        This function evaluates the data with the current model. 
        data contains both documents and labels. 
        It calls classify() to make predictions, 
        and compares with the correct labels to return 
        the model accuracy on "data". 
        '''
        num_errors = 0
        # compares target to pred and calculates error rate
        for target, note, pred in classif_dat:

            if pred != target: 
                num_errors += 1
        pred_error = num_errors/len(classif_dat) 
        
        return (1 - pred_error)
    
    
    def train_model(self, train_data, dev_data, l):
        """
        This function processes the entire training set for multiple epochs.
        After each training epoch, evaluate your model on the DEV set. 
        Save the best performing model on the DEV set to best_model
        """          

        best_acc = 0
        for i in range(self.num_epochs):
            self.train_epoch(train_data, l)
            classif, cuml_loss = self.classify(dev_data, l)
            acc = self.evaluate_classifier_accuracy(classif)
            print('epoch: {}; accuracy: {}; NLLLoss: {}'.format(i, acc, cuml_loss))
            if acc > best_acc:
                best_acc = acc 
                self.best_model['model'] = copy.deepcopy(self)
                self.best_model['acc'] = acc

In [54]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import torch.autograd as autograd

# https://github.com/claravania/lstm-pytorch/blob/master/model.py
torch.manual_seed(1)

class BiDirLSTMRNN(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, output_size):
        super(BiDirLSTMRNN, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,batch_first=True, bidirectional=True)
        self.hidden2cat = nn.Linear(hidden_dim, output_size)
        self.hidden = self.initHidden()
        self.optimizer = torch.optim.Adam(self.parameters())

        # From HW
        self.loss_fn = nn.NLLLoss(size_average=False)
        self.num_epochs = 15
        self.best_model = {}
        
    def initHidden(self):
        '''
        Sets the size and expected output for the hidden layer
        '''
        return (autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)),
                autograd.Variable(torch.randn(2, 1, self.hidden_dim // 2)))
      
    def forward(self, sentence):
        '''
        Forward pass
        '''
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        
        # self.hidden[0][-1] is the last hidden state of the sequences
        out_space = self.hidden2cat(torch.cat((self.hidden[0][0], self.hidden[0][1]), 0)) # MODIFICATION TO VERIFY W AMITABH - TRY ADDING AND TRY CONCATENATING
        
        # softmax for output layer. 
        output = F.log_softmax(out_space )
        
        return output

    def train_epoch(self, train_data, l):
        '''
        Trains an epoch. 
        '''
        data = TextClassificationDataset(train_data, l)
        data = tud.DataLoader(data, batch_size = 1, shuffle = True)
        # Forward pass
            
        for sent, targets in data: 
            self.zero_grad() 
            self.hidden = self.initHidden()
            pred = self.forward(sent)
            
            # Calculate loss with prediction and target labels

            loss = self.loss_fn(pred, targets[0][0])

            # Backward pass
            loss.backward()
            
            # Apply optimizer step. 
            self.optimizer.step()
            
    def classify(self, val_data, l):
        '''
        This function classifies/predicts documents into their categories. 
        '''
        classif = []
        cuml_loss = 0
        
        data = TextClassificationDataset(val_data, l)
        data = tud.DataLoader(data, batch_size = 1, shuffle = True)
        with torch.no_grad():
            for sent, targets in data: 
                self.hidden = self.initHidden()
                pred = self.forward(sent)

                # Calculate loss with prediction and target labels
                loss = self.loss_fn(pred, targets[0][0])
                cuml_loss += loss

                # fingin the max label for accuracy
                pred_idx = ((torch.topk(pred, 1)[1])[0]).item()
                
                pred_rel = l.idx_to_label[pred_idx]

                # storing into output
                targ_rel = l.idx_to_label[(targets[0][0].item())]
                classif.append((targ_rel, sent, pred_rel))

        return classif, cuml_loss            
                
                
    def evaluate_classifier_accuracy(self, classif_dat):
        '''
        This function gets the accuracy of classified data. 
        '''
        num_errors = 0
        # compares target to pred and calculates error rate
        for target, note, pred in classif_dat:
            if pred != target: 
                num_errors += 1
        pred_error = num_errors/len(classif_dat) 
        
        return (1 - pred_error)
    
    
    def train_model(self, train_data, dev_data, l):
        """
        This function processes the entire training set for multiple epochs.
        After each training epoch, evaluate your model on the DEV set. 
        Save the best performing model on the DEV set to best_model
        """          

        best_acc = 0
        for i in range(self.num_epochs):
            self.train_epoch(train_data, l)
            classif, cuml_loss = self.classify(dev_data, l)
            acc = self.evaluate_classifier_accuracy(classif)
            print('epoch: {}; accuracy: {}; NLLLoss: {}'.format(i, acc, cuml_loss))
            if acc > best_acc:
                best_acc = acc 
                self.best_model['model'] = copy.deepcopy(self)
                self.best_model['acc'] = acc

IndentationError: expected an indented block (<ipython-input-54-0a4c8599d7ba>, line 31)

In [60]:
EMB_DIM = 10
HID_DIM = 40
labs = labels(all_list)
tm = textModel(small_train)
tm.vocab_size

296

In [28]:
model_lstm = LSTMRNN(EMB_DIM, HID_DIM, tm.vocab_size, labs.labels_size)

model_lstm.train_model(train_list, dev_list, labs)

/Users/crismacg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0; accuracy: 0.6238425925925926; NLLLoss: 6555.48486328125
epoch: 1; accuracy: 0.7194444444444444; NLLLoss: 4942.01611328125
epoch: 2; accuracy: 0.7328703703703704; NLLLoss: 4683.599609375
epoch: 3; accuracy: 0.7347222222222223; NLLLoss: 4736.33349609375
epoch: 4; accuracy: 0.7361111111111112; NLLLoss: 4741.58251953125
epoch: 5; accuracy: 0.731712962962963; NLLLoss: 4796.4287109375
epoch: 6; accuracy: 0.7395833333333333; NLLLoss: 4820.32568359375
epoch: 7; accuracy: 0.7328703703703704; NLLLoss: 4873.9833984375
epoch: 8; accuracy: 0.7293981481481482; NLLLoss: 5071.0595703125
epoch: 9; accuracy: 0.7268518518518519; NLLLoss: 5071.08349609375
epoch: 10; accuracy: 0.7256944444444444; NLLLoss: 5168.14697265625
epoch: 11; accuracy: 0.7238425925925926; NLLLoss: 5261.71826171875
epoch: 12; accuracy: 0.7199074074074074; NLLLoss: 5292.1435546875
epoch: 13; accuracy: 0.725; NLLLoss: 5414.49462890625
epoch: 14; accuracy: 0.7231481481481481; NLLLoss: 5343.77001953125
epoch: 15; accuracy: 0.72

In [61]:
# Running 50 epochs of bidirectional LSTM 
model_bd = BiDirLSTMRNN(EMB_DIM, HID_DIM, tm.vocab_size, labs.labels_size)

model_bd.train_model(train_list, dev_list, labs)
# ----- code for tuning 

/Users/crismacg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0; accuracy: 0.6287037037037038; NLLLoss: 6352.1689453125
epoch: 1; accuracy: 0.7027777777777777; NLLLoss: 5298.0634765625
epoch: 2; accuracy: 0.7159722222222222; NLLLoss: 5062.89013671875
epoch: 3; accuracy: 0.7247685185185185; NLLLoss: 4995.65087890625
epoch: 4; accuracy: 0.7224537037037038; NLLLoss: 4906.1376953125
epoch: 5; accuracy: 0.7238425925925926; NLLLoss: 4966.78271484375
epoch: 6; accuracy: 0.7268518518518519; NLLLoss: 4904.794921875
epoch: 7; accuracy: 0.724537037037037; NLLLoss: 4979.283203125
epoch: 8; accuracy: 0.7287037037037036; NLLLoss: 5054.5546875
epoch: 9; accuracy: 0.7243055555555555; NLLLoss: 5032.3671875
epoch: 10; accuracy: 0.73125; NLLLoss: 5053.33935546875
epoch: 11; accuracy: 0.7256944444444444; NLLLoss: 5076.46728515625
epoch: 12; accuracy: 0.7182870370370371; NLLLoss: 5215.724609375
epoch: 13; accuracy: 0.7261574074074074; NLLLoss: 5149.39404296875
epoch: 14; accuracy: 0.7261574074074074; NLLLoss: 5105.72412109375


In [77]:
# ----------------------- Test LSTM 
classif, loss = model_lstm.best_model['model'].classify(test_list, labs)
print('accuracy according to validation set {}'.format(model_lstm.best_model['acc']))
print('accuracy according to test set {}'.format(model_lstm.evaluate_classifier_accuracy(classif))

/Users/crismacg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [80]:
# ------------------------ Test Bidirectional LSTM 
classif, loss = model_bd.best_model['model'].classify(test_list, labs)
print('accuracy according to validation set {}'.format(model_bd.best_model['acc']))
print('accuracy according to test set {}'.format(model_bd.evaluate_classifier_accuracy(classif))

/Users/crismacg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
# ----------------------------------------------------------------------

In [69]:
tuning_models = LSTMRNN(EMB_DIM, HID_DIM, tm.vocab_size, labs.labels_size)

In [ ]:
# TUNING THE MODEL 
# Option 1: 
results = {}

optimizers = ['Adam', 'SGD']
lrates = [0.0001, 0.01]
regularization = [0, .3]
epochs = [8]
count = 0
for optim in optimizers:
    results[optim] = {}
    for rate in lrates: 
        results[optim][rate] = {}
        for regul in regularization: 
            results[optim][rate][regul] = {}
            for n_epochs in epochs: 
                count += 1
                tuning_models.num_epochs = n_epochs
                tuning_models.train_model(train_list, dev_list, labs)
                if optim == 'Adam': 
                    tuning_models.optimizer = torch.optim.Adam(tuning_models.parameters(), lr = rate, weight_decay = regul)
                if optim == 'SGD': 
                    tuning_models.optimizer = torch.optim.SGD(tuning_models.parameters(), lr = rate, weight_decay = regul)
                
                results[optim][rate][regul]['best_m'] = tuning_models.best_model['model']
                results[optim][rate][regul]['acc'] = tuning_models.best_model['acc']
                print('Iteration {} completed'.format(count))

/Users/crismacg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0; accuracy: 0.7108796296296296; NLLLoss: 5860.40673828125
epoch: 1; accuracy: 0.7083333333333333; NLLLoss: 5993.873046875
epoch: 2; accuracy: 0.7118055555555556; NLLLoss: 5869.85791015625
epoch: 3; accuracy: 0.7074074074074075; NLLLoss: 5872.5068359375
epoch: 4; accuracy: 0.7078703703703704; NLLLoss: 5873.8330078125
epoch: 5; accuracy: 0.7125; NLLLoss: 5787.69580078125
epoch: 6; accuracy: 0.7076388888888889; NLLLoss: 5981.404296875
epoch: 7; accuracy: 0.7064814814814815; NLLLoss: 5915.35595703125
Iteration 1 completed
epoch: 0; accuracy: 0.7141203703703703; NLLLoss: 5916.125
epoch: 1; accuracy: 0.712962962962963; NLLLoss: 5951.59033203125
epoch: 2; accuracy: 0.7111111111111111; NLLLoss: 5972.73828125
epoch: 3; accuracy: 0.7111111111111111; NLLLoss: 5996.56201171875
epoch: 4; accuracy: 0.7097222222222221; NLLLoss: 6031.03076171875
epoch: 5; accuracy: 0.7074074074074075; NLLLoss: 6053.3623046875
epoch: 6; accuracy: 0.7101851851851853; NLLLoss: 6060.77734375
epoch: 7; accuracy: 0.

In [ ]:
# SELECTING BEST MODEL 

best = {}
best['best_acc'] = 0
best['best_mod'] = None
for optim in optimizers:
    for rate in lrates: 
        for regul in regularization: 
            for n_epochs in epochs: 
                if results[optim][rate][regul]['acc'] > best['best_acc']:                     
                    best['best_acc'] = results[optim][rate][regul]['acc']
                    print(results[optim][rate][regul])
                    best['best_mod'] = results[optim][rate][regul]['best_m']
                    best['n_epochs'] = n_epochs 

In [ ]:
# IDENTIFY WHERE WE ARE MAKING MORE MISTAKES 
results = pd.DataFrame(classif, usecols = ['tar', 'sent', 'pred'])
results['correct'] = results.apply(lambda x: 1 if x['tar'] == x['pred'] else 0)
results.groupby('tar')['correct'].sum()

In [ ]:
#### Where are we making mistakes 
results = pd.DataFrame(classif)